<a href="https://colab.research.google.com/github/independentmisfit/Speechsense/blob/main/Speechsenseipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import gradio as gr
import whisper
import numpy as np
from langdetect import detect
from gtts import gTTS
from transformers import pipeline
from deep_translator import GoogleTranslator  # Updated import for translation
import os

In [ ]:
 Function to map sentiment to emojis
def get_sentiment_emoji(sentiment):
    emoji_mapping = {
        "disappointment": "😞",
        "sadness": "😢",
        "annoyance": "😠",
        "neutral": "😐",
        "disapproval": "👎",
        "realization": "😮",
        "nervousness": "😬",
        "approval": "👍",
        "joy": "😄",
        "anger": "😡",
        "embarrassment": "😳",
        "caring": "🤗",
        "remorse": "😔",
        "disgust": "🤢",
        "grief": "😥",
        "confusion": "😕",
        "relief": "😌",
        "desire": "😍",
        "admiration": "😌",
        "optimism": "😊",
        "fear": "😨",
        "love": "❤️",
        "excitement": "🎉",
        "curiosity": "🤔",
        "amusement": "😄",
        "surprise": "😲",
        "gratitude": "🙏",
        "pride": "🦁"
    }
    return emoji_mapping.get(sentiment, "")
"""


In [ ]:

title = """# 🎙️ SpeechSense: Transcription, Emotion Detection, Translation & Voice Playback 🌍

In [ ]:
# Description for the app
description = description = """App Description:
SpeechSense uses cutting-edge technologies to process audio efficiently. It employs OpenAI’s Whisper model for accurate transcription, DistilBERT for emotion detection, LangDetect for language identification, Google Translate API for multilingual translation, and gTTS for natural text-to-speech conversion. Together, these models enable real-time speech-to-text, emotion recognition, translation, and voice playback in multiple languages., and then it works its magic by:
-Transcribing your speech into text 📝.
-Detecting the language of your spoken words 🌐.
-Recognizing emotions in your voice, like happiness, sadness, or excitement 😄😢😠.
-Translating the transcribed text into a language of your choice 🎯.
-Playing back the translated text as audio using natural-sounding voices 🔊.
-Whether you’re looking to translate a conversation, identify emotions in speech, or just hear your words in another language, SpeechSense simplifies it all for you!
How to Use the App:
Step-by-Step Guide:
-Choose Audio Input Type 🎤:
  -Select how you want to provide audio:
    -Record Audio: Speak directly into your microphone.
    -Upload Audio: Upload an existing audio file (MP3 or WAV format).
-Record or Upload Your Audio 🎧:
    -If recording, click the record button and speak.
    -If uploading, click to browse your device and select an audio file.
-Select a Language for Translation 🌍:
    -From the dropdown, choose the language you want your speech to be translated into. For example, you can select Spanish, French, Chinese, or any other language!
-Process the Audio 🖱️:
  -Click the Process Audio button. The app will:
  -Transcribe your audio into text.
  -Detect the language you spoke.
  -Analyze and identify any emotions in the speech.
  -Translate the transcription into your chosen language.
  -Convert the translated text into speech and play it back!
-Review and Listen 🎶:
  -You’ll see:
    -Transcribed Text: The text version of your speech.
    -Detected Language: The language spoken in the audio.
    -Recognized Emotion: The dominant emotion in your speech.
    -Translated Text: The text translated into your selected language.
    -Audio Output: Listen to the translation read aloud!
-Key Features:
-Dual Input: Record audio or upload an audio file for transcription and processing.
-Speech-to-Text: Accurately converts speech into written text.
-Language Identification: Detects the language of the spoken words.
-Emotion Detection: Recognizes emotions such as joy, sadness, and more.
-Multilingual Translation: Supports translation into numerous languages.
-Text-to-Speech (TTS): Reads out the translated text in a natural-sounding voice.
-SpeechSense is your go-to tool for real-time speech transcription, emotion detection, multilingual translation, and audio playback—all in one place!"""


In [ ]:
# Initialize models
whisper_model = whisper.load_model("base")
sentiment_pipeline = pipeline("sentiment-analysis", framework="pt", model="SamLowe/roberta-base-go_emotions")
translator_instance = GoogleTranslator()
language_options = translator_instance.get_supported_languages(as_dict=True)
language_display = {name.title(): code for name, code in language_options.items()}


# Function to process audio using Whisper and deep-translator
def process_audio(audio, selected_language_name):
    target_language = language_display[selected_language_name]

    # Check if file was uploaded or recorded
    if isinstance(audio, dict):
        audio_file_path = audio['name']  # Get file path of uploaded file
    else:
        audio_file_path = audio  # Handle recorded audio case

    # Transcribe the audio using Whisper
    result = whisper_model.transcribe(audio_file_path)
    text = result['text']

    # Detect the language of the text
    detected_language = detect(text)

    # Sentiment analysis
    sentiment = sentiment_pipeline(text)[0]
    sentiment_label = sentiment['label']
    sentiment_emoji = get_sentiment_emoji(sentiment_label)

    # Translate the text using Deep Translator
    translated_text = GoogleTranslator(source='auto', target=target_language).translate(text)
    # Language options for translation
    language_options = GoogleTranslator.get_supported_languages(as_dict=True)
    language_display = {name.title(): code for code, name in language_options.items()}



    # Convert the translated text to speech
    tts = gTTS(translated_text, lang=target_language)
    tts_file = "output_audio.mp3"
    tts.save(tts_file)

    return text, detected_language, sentiment_emoji, sentiment['score'], translated_text, tts_file


In [ ]:
# Gradio interface
with gr.Blocks() as interface:
    gr.Markdown(title)
    gr.Markdown(description)

    # Select whether to record or upload audio
    audio_input_type = gr.Radio(["Record Audio", "Upload Audio"], label="Choose Audio Input Type", value="Record Audio")

    # Set up recording and upload input options
    with gr.Row():
        audio_record = gr.Audio(label="Record your audio", type="filepath", visible=True, sources="microphone")
        audio_upload = gr.File(type="filepath", label="Upload an audio file (MP3/WAV)", file_types=["audio/mp3", "audio/wav"], visible=False)


    # Language selection
    target_language = gr.Dropdown(choices=list(language_display.keys()), value="English", label="Select language for translation", interactive=True)



    # Process button
    submit_button = gr.Button("Process Audio")

    # Outputs for transcription, language, sentiment, translation, and audio
    transcription_output = gr.Textbox(label="Transcribed Text")
    detected_language_output = gr.Textbox(label="Detected Language")
    sentiment_output = gr.Textbox(label="Sentiment")
    sentiment_score_output = gr.Textbox(label="Sentiment Score")
    translated_text_output = gr.Textbox(label="Translated Text")
    audio_output = gr.Audio(label="Translated Audio")

    # Function to toggle between audio inputs
    def update_audio_input(input_type):
        return gr.update(visible=input_type == "Record Audio"), gr.update(visible=input_type == "Upload Audio")

    # Handle audio input processing
    def process_based_on_input(input_type, record_audio, upload_audio, selected_language):
        if input_type == "Record Audio" and record_audio is not None:
            audio_file = record_audio
        elif input_type == "Upload Audio" and upload_audio is not None:
            audio_file = upload_audio.name
        else:
            return "No audio file provided.", None, None, None, None

        # Process the audio and return the outputs
        return process_audio(audio_file, selected_language)

    # Link the inputs to the function
    audio_input_type.change(fn=update_audio_input, inputs=audio_input_type, outputs=[audio_record, audio_upload])
    submit_button.click(fn=process_based_on_input, inputs=[audio_input_type, audio_record, audio_upload, target_language], outputs=[transcription_output, detected_language_output, sentiment_output, sentiment_score_output, translated_text_output, audio_output])

# Launch the interface
interface.launch(share=True)